In [1]:
delete from [Theme_Park].[Maintenance_Calendar]
go
drop table [Theme_Park].[Maintenance_Notification]
go
create table [Theme_Park].[Maintenance_Notification]
(
    notification_id int identity(1,1),
    maintenance_id int not null,
    completion datetime not null,
    billed_hours int,
    amount smallmoney,
)

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the table 'Theme_Park.Maintenance_Notification', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.121

## Trigger TRIGGER_MANAGER_NOTIFICATION_ON_MAINTENANCE_COMPLETION:

This trigger is intended to create notification entries whenever a maintenance event is updated or inserted
with a completion time set. These notifications have 


In [2]:
drop trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
go
create trigger [Theme_Park].[Trigger_Manager_Notification_On_Maintenance_Completion]
on [Theme_Park].[Maintenance_Calendar]
after insert,update
as
BEGIN
    set nocount on
    declare 
        @id int,
        @Completion_Date datetime,
        @Hours int,
        @Amount smallmoney
    declare cur cursor read_only for 
        select Maint_ID, Maint_Completion, Billed_Hours, Invoice_amount from inserted
    open cur
    fetch next from cur into @id, @Completion_Date, @Hours, @Amount
    while @@fetch_status = 0
    begin
        if (@Completion_Date is not null)
        begin
            insert into [Theme_Park].[Maintenance_Notification]
            values (@id, @Completion_Date, @Hours, @Amount)
        end
        fetch next from cur into @id, @Completion_Date, @Hours, @Amount
    end
    close cur
    deallocate cur
end
    

: Msg 3701, Level 11, State 5, Line 1
Cannot drop the trigger 'Theme_Park.Trigger_Manager_Notification_On_Maintenance_Completion', because it does not exist or you do not have permission.

Commands completed successfully.

Total execution time: 00:00:00.101

In [4]:
insert into [Theme_Park].[Maintenance_Calendar]
(Occurence_datetime, Priority, Maint_Start, Maint_Completion, Billed_Hours, Invoice_amount)
values
(
    GETDATE(), 'ASAP', GETDATE(), GETDATE(), 1, 1130.50 
)

(1 row affected)

Total execution time: 00:00:00.061

In [8]:
insert into [Theme_Park].[Maintenance_Calendar]
(Occurence_datetime, Priority, Maint_Start)
values
(
    GETDATE(), 'ASAP', GETDATE()
)

(1 row affected)

Total execution time: 00:00:00.053

In [12]:
update [Theme_Park].[Maintenance_Calendar]
set Maint_Completion = GETDATE()
where Maint_ID = 3
go

(1 row affected)

Total execution time: 00:00:00.058

In [28]:
select * from [Theme_Park].[Maintenance_Notification]

(10 rows affected)

Total execution time: 00:00:00.137

notification_id,maintenance_id,completion,billed_hours,amount
1,1,2022-11-02 18:22:44.457,1,1130.50
2,2,2022-11-02 18:22:50.080,1,1130.50
3,3,2022-11-02 18:26:44.707,NULL,NULL
4,3,2022-11-02 18:26:53.157,NULL,NULL
5,13,2022-11-03 00:47:21.440,1,1.00
6,13,2022-11-03 00:48:10.653,1,1.00
7,13,2022-11-03 00:48:28.907,1,1.00
8,13,2022-11-03 00:48:48.937,1,1.00
9,13,2022-11-03 00:49:16.653,2,2.00
10,13,2022-11-03 00:50:01.873,2,420.69


In [16]:
declare @date datetime = GETDATE()
exec [Theme_Park].[Proc_Add_Maint] @date, null, null, 'Urgent', @date
go 

(1 row affected)

Total execution time: 00:00:00.086

In [26]:
select * from [Theme_Park].[Maintenance_Calendar]
go
select * from [Theme_Park].[Attractions]
GO
select * from [Theme_Park].[Vendor]
go
select * from [Theme_Park].[Department]
go

(7 rows affected)

(1 row affected)

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.313

Maint_ID,Occurence_datetime,Vendor_ID,Attraction_ID,Priority,Maint_Start,Maint_Completion,Billed_Hours,Invoice_amount,Scanned_invoice
1,2022-11-02 18:22:44.457,NULL,NULL,ASAP,2022-11-02 18:22:44.457,2022-11-02 18:22:44.457,1,1130.50,NULL
2,2022-11-02 18:22:50.080,NULL,NULL,ASAP,2022-11-02 18:22:50.080,2022-11-02 18:22:50.080,1,1130.50,NULL
3,2022-11-02 18:25:32.470,NULL,NULL,ASAP,2022-11-02 18:25:32.470,2022-11-02 18:26:53.157,NULL,NULL,NULL
4,2022-11-02 18:25:33.410,NULL,NULL,ASAP,2022-11-02 18:25:33.410,NULL,NULL,NULL,NULL
5,2022-11-02 18:25:34.140,NULL,NULL,ASAP,2022-11-02 18:25:34.140,NULL,NULL,NULL,NULL
6,2022-11-02 23:20:44.100,NULL,NULL,Urgent,2022-11-02 23:20:44.100,NULL,NULL,NULL,NULL
7,2022-11-02 23:23:34.757,NULL,NULL,Urgent,2022-11-02 23:23:34.757,NULL,NULL,NULL,NULL


attraction_id,name,dept_id
1,Spinny Spin,6


name,vendor_point_of_contact_name,business_phone,business_address,service_type,contract_start_date,scanned_contract,vendor_id
Bills Sandwiches,Jim Bob,5125125125,"123 Sandwich Park, Houston TX 77004",Food,2022-09-12,NULL,2


dept_id,dept_name,supervisor_ssn
3,Admin,NULL
4,Research,NULL
5,Food Court,NULL
6,Thrills,NULL
